In [39]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import yfinance as yf
from tiny_ta import atr, rsi, sma

# Reversal - Strategie im S&P 500

Zunächst werden die Kursdaten für den S&P 500 geladen und bereinigt

In [40]:
SYMBOL = "^GSPC"
stock = yf.download(SYMBOL)

stock = stock[~(stock.High == stock.Low) & ~(stock.Open == stock.Close)]
stock = stock.dropna()

[*********************100%%**********************]  1 of 1 completed


Ein paar Indikatoren für spätere Auswertungen

In [41]:
stock["rsi_2"] = rsi(stock.Close, 2)
stock["rsi_3"] = rsi(stock.Close, 3)
stock["rsi_7"] = rsi(stock.Close, 7)
stock["rsi_10"] = rsi(stock.Close, 10)

stock["atr_10"] = atr(stock, 10, smoothing="rma") / stock.Close * 100

adx_intervall = 21
adx = ta.adx(stock.High, stock.Low, stock.Close, adx_intervall)
stock["adx"] = adx[f"ADX_{adx_intervall}"]
stock["adx_direction"] = np.where(
    adx[f"DMP_{adx_intervall}"] > adx[f"DMN_{adx_intervall}"], 1, -1
)

stock["adx_10"] = ta.adx(stock.High, stock.Low, stock.Close, 10, 10)["ADX_10"]


stock["sma"] = sma(stock.Close, 20)
stock["sma_20"] = stock.Close / sma(stock.Close, 20)
stock["sma_50"] = stock.Close / sma(stock.Close, 50)
stock["sma_100"] = stock.Close / sma(stock.Close, 100)
stock["sma_200"] = stock.Close / sma(stock.Close, 200)

stock["sma_pct"] = (stock.Close / stock.sma).round(2)

stock["month"] = stock.index.month

df = stock

## Jetzt zur Strategie
Es wird gekauft wenn der RSI unter 30 gesprungen ist und bei einem RSI über 50 wieder verkauft

In [42]:
from backtesting import Strategy


class reversal(Strategy):
    """
    strategy for reversals
    """

    rsi_exit = 50
    rsi_before = 30
    rsi_entry = 15

    def ohlc(self, pos=-1) -> tuple:
        """
        helper function for ohlc data
        """
        return (
            self.data.Open[pos],
            self.data.High[pos],
            self.data.Low[pos],
            self.data.Close[pos],
        )

    def init(self):
        super().init()

        self.rsi = self.I(lambda: self.data.rsi_2, name="RSI(2)", overlay=False)

    def next(self):
        # super().next()

        _open, _high, _low, _close = self.ohlc()

        # trade management for an existing trade
        for trade in self.trades:
            if trade.is_long:
                if self.rsi[-1] >= self.rsi_exit:
                    trade.close()

        if self.rsi[-1] <= self.rsi_entry and self.rsi[-2] <= self.rsi_before:
            self.buy()

In [43]:
from backtesting import Backtest

bt = Backtest(
    df["2012-01-01":], reversal, cash=100_000, commission=0.002, trade_on_close=True
)
stats = bt.run()
bt.plot(superimpose=False)  # , open_browser=False)
stats

Start                     2012-01-03 00:00:00
End                       2024-04-15 00:00:00
Duration                   4486 days 00:00:00
Exposure Time [%]                    18.74393
Equity Final [$]                152432.239918
Equity Peak [$]                 155157.099001
Return [%]                           52.43224
Buy & Hold Return [%]              296.365057
Return (Ann.) [%]                    3.498814
Volatility (Ann.) [%]                8.703509
Sharpe Ratio                            0.402
Sortino Ratio                        0.661492
Calmar Ratio                         0.222418
Max. Drawdown [%]                  -15.730783
Avg. Drawdown [%]                   -2.554973
Max. Drawdown Duration     1722 days 00:00:00
Avg. Drawdown Duration      121 days 00:00:00
# Trades                                  161
Win Rate [%]                        72.670807
Best Trade [%]                       9.164445
Worst Trade [%]                     -4.867701
Avg. Trade [%]                    

In [44]:
stats, heatmap = bt.optimize(
    rsi_entry=range(4, 15),
    rsi_exit=range(20, 50, 1),
    rsi_before=range(5, 30, 1),
    maximize="Profit Factor",  # "Equity Final [$]",
    max_tries=200,
    random_state=0,
    return_heatmap=True,
)

/Users/produktmanagement/Python/github/backtesting-trading-strategies/.venv/lib/python3.12/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


In [45]:
# heatmap
heatmap.sort_values().iloc[-3:]

rsi_entry  rsi_exit  rsi_before
8          30        8             2.280291
7          28        9             2.297775
6          44        9             2.359036
Name: Profit Factor, dtype: float64

In [46]:
heatmap.groupby(["rsi_entry", "rsi_exit", "rsi_before"]).mean().unstack()

rsi_before                5   6   7         8         9        10  11  12  13  \
rsi_entry rsi_exit                                                              
4         20             NaN NaN NaN  1.644873       NaN      NaN NaN NaN NaN   
          23             NaN NaN NaN       NaN       NaN      NaN NaN NaN NaN   
          24             NaN NaN NaN       NaN       NaN      NaN NaN NaN NaN   
          26        1.465571 NaN NaN       NaN       NaN      NaN NaN NaN NaN   
          27             NaN NaN NaN       NaN       NaN      NaN NaN NaN NaN   
...                      ...  ..  ..       ...       ...      ...  ..  ..  ..   
14        38             NaN NaN NaN       NaN  1.941504  1.64869 NaN NaN NaN   
          40             NaN NaN NaN       NaN       NaN      NaN NaN NaN NaN   
          45             NaN NaN NaN       NaN       NaN      NaN NaN NaN NaN   
          46             NaN NaN NaN       NaN       NaN      NaN NaN NaN NaN   
          49        2.259054 NaN NaN       NaN       NaN      NaN NaN NaN NaN   

rsi_before          14  ...  20        21        22  23  24        25  \
rsi_entry rsi_exit      ...                                             
4         20       NaN  ... NaN       NaN       NaN NaN NaN       NaN   
          23       NaN  ... NaN       NaN  1.131931 NaN NaN       NaN   
          24       NaN  ... NaN       NaN       NaN NaN NaN       NaN   
          26       NaN  ... NaN       NaN       NaN NaN NaN       NaN   
          27       NaN  ... NaN       NaN       NaN NaN NaN       NaN   
...                 ..  ...  ..       ...       ...  ..  ..       ...   
14        38       NaN  ... NaN       NaN       NaN NaN NaN       NaN   
          40       NaN  ... NaN       NaN       NaN NaN NaN  1.371386   
          45       NaN  ... NaN  1.246496       NaN NaN NaN       NaN   
          46       NaN  ... NaN       NaN       NaN NaN NaN       NaN   
          49       NaN  ... NaN       NaN       NaN NaN NaN       NaN   

rsi_before                26  27  28  29  
rsi_entry rsi_exit                        
4         20             NaN NaN NaN NaN  
          23             NaN NaN NaN NaN  
          24             NaN NaN NaN NaN  
          26             NaN NaN NaN NaN  
          27        1.070347 NaN NaN NaN  
...                      ...  ..  ..  ..  
14        38             NaN NaN NaN NaN  
          40             NaN NaN NaN NaN  
          45             NaN NaN NaN NaN  
          46             NaN NaN NaN NaN  
          49             NaN NaN NaN NaN  

[154 rows x 25 columns]

In [47]:
stats

Start                     2012-01-03 00:00:00
End                       2024-04-15 00:00:00
Duration                   4486 days 00:00:00
Exposure Time [%]                    6.895435
Equity Final [$]                 131488.51898
Equity Peak [$]                  131488.51898
Return [%]                          31.488519
Buy & Hold Return [%]              296.365057
Return (Ann.) [%]                    2.258365
Volatility (Ann.) [%]                5.143412
Sharpe Ratio                         0.439079
Sortino Ratio                        0.695829
Calmar Ratio                          0.25572
Max. Drawdown [%]                   -8.831401
Avg. Drawdown [%]                   -2.282351
Max. Drawdown Duration     1047 days 00:00:00
Avg. Drawdown Duration      118 days 00:00:00
# Trades                                   63
Win Rate [%]                        76.190476
Best Trade [%]                       2.510704
Worst Trade [%]                     -3.692599
Avg. Trade [%]                    

The Winner is ...

In [48]:
stats["_strategy"]

<Strategy reversal(rsi_entry=6,rsi_exit=44,rsi_before=9)>

In [49]:
stats = bt.run(rsi_entry=6, rsi_exit=44, rsi_before=9)
bt.plot(superimpose=False, open_browser=False)

Row(id='7250', ...)

In [50]:
stats

Start                     2012-01-03 00:00:00
End                       2024-04-15 00:00:00
Duration                   4486 days 00:00:00
Exposure Time [%]                    6.895435
Equity Final [$]                 131488.51898
Equity Peak [$]                  131488.51898
Return [%]                          31.488519
Buy & Hold Return [%]              296.365057
Return (Ann.) [%]                    2.258365
Volatility (Ann.) [%]                5.143412
Sharpe Ratio                         0.439079
Sortino Ratio                        0.695829
Calmar Ratio                          0.25572
Max. Drawdown [%]                   -8.831401
Avg. Drawdown [%]                   -2.282351
Max. Drawdown Duration     1047 days 00:00:00
Avg. Drawdown Duration      118 days 00:00:00
# Trades                                   63
Win Rate [%]                        76.190476
Best Trade [%]                       2.510704
Worst Trade [%]                     -3.692599
Avg. Trade [%]                    

Die Win Rate beim Trading bezieht sich auf das Verhältnis der Anzahl der gewonnenen Trades zur Gesamtanzahl der Trades. 
Wenn jemand beispielsweise 70% seiner Trades gewinnt, beträgt seine Win Rate 70%.

In [51]:
stats["Win Rate [%]"]

76.19047619047619

Durchschnittlicher Gewinn in Prozent. 
Wie viel Gewinn oder Verlust wurden im Durchschnitt pro Trade erzielt.

In [52]:
stats["Avg. Trade [%]"]

0.4446708191502502

Der Profit-Faktor stellt die Verhältnisse zwischen Gewinnen und Verlusten dar.
Der Profit-Faktor wird berechnet, indem die Summe aller Profite (Gewinne) durch die Summe aller Verluste (einschließlich Gebühren und Kommissionen) für den gesamten Handelszeitraum geteilt wird.
Ein Profit-Faktor größer als 1 zeigt an, dass die Strategie profitabel ist. Je höher der Wert, desto besser.

Ein guter Profit-Faktor liegt normalerweise zwischen 1,75 und 41.
Ein Wert über 1,75 zeigt eine solide Strategie an, während Werte über 4 außergewöhnlich gut sind.

In [53]:
stats["Profit Factor"]

2.3590362198439156

In [54]:
trades = stats["_trades"]

for index, row in trades.iterrows():
    start_range = stock[: row["EntryTime"]]
    trades.loc[index, "SignalTime"] = start_range.index.values[-2]

merged = pd.merge(
    trades,
    df[
        [
            "adx_10",
            "adx_direction",
            "rsi_3",
            "rsi_7",
            "rsi_10",
            "sma_20",
            "sma_50",
            "sma_100",
            "sma_200",
        ]
    ],
    left_on="SignalTime",
    right_index=True,
)
# merged["win"] = np.where(merged.PnL > 0, 1, -1)
# merged["win"] = np.where(merged.PnL > 0, 1, -1)

In [55]:
trades = stats["_trades"]

for index, row in trades.iterrows():
    start_range = stock[: row["EntryTime"]]
    trades.loc[index, "SignalTime"] = start_range.index.values[-2]

merged = pd.merge(
    trades,
    df[
        [
            "adx_10",
            "adx_direction",
            "rsi_3",
            "rsi_7",
            "rsi_10",
            "sma_20",
            "sma_50",
            "sma_100",
            "sma_200",
        ]
    ],
    left_on="SignalTime",
    right_index=True,
)

In [56]:
pd.DataFrame(merged.groupby(merged.ExitTime.dt.year).PnL.sum().round(1))

,PnL
ExitTime,
2012,3944.8
2013,360.3
2014,3769.1
2015,-20.3
2016,29.4
2017,708.5
2018,-1655.5
2019,3053.4
2020,-626.0


In [57]:
equity = stats["_equity_curve"]
equity.groupby(equity.index.year).Equity.last().round(1)

2012    103944.8
2013    104305.1
2014    108074.2
2015    108053.9
2016    108083.4
2017    108791.9
2018    107136.4
2019    110189.7
2020    109563.7
2021    113266.6
2022    124432.9
2023    129720.5
2024    131488.5
Name: Equity, dtype: float64